<a href="https://colab.research.google.com/github/sankardevisharath/amex-default-prediction/blob/master/notebooks/svm_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%mkdir data
%cd data
%mkdir raw
%cd raw

/content/data
/content/data/raw


In [3]:
!cp /content/drive/MyDrive/amex-default-prediction/data/raw/data_2017-03.parquet /content/data/raw/

In [4]:
!unzip /content/drive/MyDrive/amex-default-prediction/data/raw/amex-default-prediction.zip train_labels.csv

Archive:  /content/drive/MyDrive/amex-default-prediction/data/raw/amex-default-prediction.zip
  inflating: train_labels.csv        


In [5]:
import numpy as np
import pandas as pd

In [6]:
TRAIN_DATA_PATH = '/content/data/raw/data_2017-03.parquet'
TRAIN_LABELS_PATH = '/content/data/raw/train_labels.csv'

In [7]:
df = pd.read_parquet(TRAIN_DATA_PATH)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395630 entries, 0 to 395629
Columns: 190 entries, customer_ID to D_145
dtypes: datetime64[ns](1), float64(185), int64(1), object(3)
memory usage: 573.5+ MB


In [9]:
df['customer_ID'].nunique()

395630

In [10]:
def read_cols(cols):
  df = pd.read_csv(TRAIN_DATA_PATH, usecols=cols)
  df = append_label(df)
  return df

def append_label(source_df):
  return pd.merge(left=source_df, right=train_labels, how='inner')

In [11]:
train_labels = pd.read_csv(TRAIN_LABELS_PATH)

In [12]:
df = append_label(df)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 395630 entries, 0 to 395629
Columns: 191 entries, customer_ID to target
dtypes: datetime64[ns](1), float64(185), int64(2), object(3)
memory usage: 579.5+ MB


In [14]:
cat_cols = ['D_63', 'D_64','B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_66', 'D_68']

In [15]:
df[cat_cols].isnull().sum() 

D_63          0
D_64      24578
B_30        199
B_38        199
D_114     14105
D_116     14105
D_117     14105
D_120     14105
D_126     14105
D_66     352773
D_68      24578
dtype: int64

In [16]:
df[cat_cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 395630 entries, 0 to 395629
Data columns (total 11 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   D_63    395630 non-null  object 
 1   D_64    371052 non-null  object 
 2   B_30    395431 non-null  float64
 3   B_38    395431 non-null  float64
 4   D_114   381525 non-null  float64
 5   D_116   381525 non-null  float64
 6   D_117   381525 non-null  float64
 7   D_120   381525 non-null  float64
 8   D_126   381525 non-null  float64
 9   D_66    42857 non-null   float64
 10  D_68    371052 non-null  float64
dtypes: float64(9), object(2)
memory usage: 36.2+ MB


In [17]:
from sklearn.preprocessing import OrdinalEncoder

In [18]:
 enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

In [19]:
df[['D_63', 'D_64']]=enc.fit_transform(df[['D_63', 'D_64']])

In [20]:
enc.categories_

[array(['CL', 'CO', 'CR', 'XL', 'XM', 'XZ'], dtype=object),
 array(['-1', 'O', 'R', 'U', None], dtype=object)]

In [21]:
df[cat_cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 395630 entries, 0 to 395629
Data columns (total 11 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   D_63    395630 non-null  float64
 1   D_64    395630 non-null  float64
 2   B_30    395431 non-null  float64
 3   B_38    395431 non-null  float64
 4   D_114   381525 non-null  float64
 5   D_116   381525 non-null  float64
 6   D_117   381525 non-null  float64
 7   D_120   381525 non-null  float64
 8   D_126   381525 non-null  float64
 9   D_66    42857 non-null   float64
 10  D_68    371052 non-null  float64
dtypes: float64(11)
memory usage: 36.2 MB


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 395630 entries, 0 to 395629
Columns: 191 entries, customer_ID to target
dtypes: datetime64[ns](1), float64(187), int64(2), object(1)
memory usage: 579.5+ MB


In [23]:
df.fillna(0, inplace=True)

In [24]:
df[cat_cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 395630 entries, 0 to 395629
Data columns (total 11 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   D_63    395630 non-null  float64
 1   D_64    395630 non-null  float64
 2   B_30    395630 non-null  float64
 3   B_38    395630 non-null  float64
 4   D_114   395630 non-null  float64
 5   D_116   395630 non-null  float64
 6   D_117   395630 non-null  float64
 7   D_120   395630 non-null  float64
 8   D_126   395630 non-null  float64
 9   D_66    395630 non-null  float64
 10  D_68    395630 non-null  float64
dtypes: float64(11)
memory usage: 36.2 MB


In [25]:
from sklearn.model_selection import train_test_split


In [26]:
X = df.loc[:, ~df.columns.isin(['customer_ID', 'S_2', 'target'])]
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 395630 entries, 0 to 395629
Columns: 188 entries, P_2 to D_145
dtypes: float64(187), int64(1)
memory usage: 570.5 MB


In [27]:
Y = df.loc[:, df.columns == 'target']
Y.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 395630 entries, 0 to 395629
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   target  395630 non-null  int64
dtypes: int64(1)
memory usage: 6.0 MB


In [28]:
 X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [29]:
X_train.shape

(316504, 188)

In [30]:
X_test.shape

(79126, 188)

In [31]:
from sklearn.svm import SVC

In [32]:
svm = SVC(kernel="linear", probability=True)

In [ ]:
svm.fit(X_train, y_train.values.ravel())

According to [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC) "The fit time scales at least quadratically with the number of samples and may be impractical beyond tens of thousands of samples. For large datasets consider using LinearSVC or SGDClassifier instead, possibly after a Nystroem transformer.". My dataset size is 400K and this method will not work.

